In [162]:
import pandas as pd
import numpy as np
import csv

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score

from xgboost import XGBClassifier

# Missing values imputation 

XGboost doesn't need categorical variables to be encoded numerically, so we keep the columns in their original form. This is why the imputation is redone here, because we end up with train, valid and test datasets which still have categories as strings and no dummies created.

In [75]:
campaign_ad = pd.read_csv("MLUnige2023_subscriptions_train.csv", index_col="Id")
campaign_test = pd.read_csv("MLUnige2023_subscriptions_test.csv", index_col="Id")

num_vars = ['age', 'time_spent', 'banner_views', 'banner_views_old', 'days_elapsed_old', 'X4']

In [76]:
X = campaign_ad.drop(columns='subscription')
y = campaign_ad['subscription']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=12)
X_valid, X_test, y_valid, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=46)

In [77]:
enc = OrdinalEncoder(categories=[['smartphone', 'desktop']])

In [78]:
X_tr_dev = X_train.drop(columns=['job', 'education'])
X_tr_imp_dev = X_tr_dev[X_tr_dev['device'] == 'na'].drop(columns='device')
X_tr_dev = X_tr_dev[X_tr_dev['device'] != 'na'] # this is the data set that does not have NAs
y_tr_dev = X_tr_dev[['device']] 
X_tr_dev = X_tr_dev.drop(columns='device')
y_tr_dev = enc.fit_transform(y_tr_dev)
y_tr_dev.shape = (y_tr_dev.shape[0],)

In [79]:
X_v_dev = X_valid.drop(columns=['job', 'education'])
X_v_imp_dev = X_v_dev[X_v_dev['device'] == 'na'].drop(columns='device')
X_v_dev = X_v_dev[X_v_dev['device'] != 'na']
y_v_dev = X_v_dev[['device']]
X_v_dev = X_v_dev.drop(columns='device')
y_v_dev = enc.fit_transform(y_v_dev)

In [80]:
X_te_dev = X_test.drop(columns=['job', 'education'])
X_te_imp_dev = X_te_dev[X_te_dev['device'] == 'na'].drop(columns='device')
X_te_dev = X_te_dev[X_te_dev['device'] != 'na']
y_te_dev = X_te_dev[['device']]
X_te_dev = X_te_dev.drop(columns='device')
y_te_dev = enc.fit_transform(y_te_dev)

In [81]:
preprocessor = ColumnTransformer(transformers=[('cat', OneHotEncoder(), ['marital', 'outcome_old'])],
                                 remainder='passthrough')

rfc_dev = RandomForestClassifier(n_estimators=100, random_state=59, n_jobs=-2)

device_imputation = Pipeline([
    ("preprocessor", preprocessor),
    ("rfc_dev", rfc_dev)
])

In [82]:
device_imputation.fit(X_tr_dev, y_tr_dev)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cat', OneHotEncoder(),
                                                  ['marital',
                                                   'outcome_old'])])),
                ('rfc_dev',
                 RandomForestClassifier(n_jobs=-2, random_state=59))])

In [83]:
y_tr_imp_dev = device_imputation.predict(X_tr_imp_dev)
y_v_imp_dev  = device_imputation.predict(X_v_imp_dev)
y_te_imp_dev = device_imputation.predict(X_te_imp_dev)

In [84]:
X_train.loc[X_tr_imp_dev.index, 'device'] = y_tr_imp_dev
X_valid.loc[X_v_imp_dev.index, 'device'] = y_v_imp_dev
X_test.loc[X_te_imp_dev.index, 'device'] = y_te_imp_dev

In [85]:
X_train.loc[X_train['device'] == 0, 'device'] = 'smartphone'
X_train.loc[X_train['device'] == 1, 'device'] = 'desktop'
X_valid.loc[X_valid['device'] == 0, 'device'] = 'smartphone'
X_valid.loc[X_valid['device'] == 1, 'device'] = 'desktop'
X_test.loc[X_test['device'] == 0, 'device'] = 'smartphone'
X_test.loc[X_test['device'] == 1, 'device'] = 'desktop'

In [86]:
imputer = SimpleImputer(missing_values="na", strategy='most_frequent')
X_train[['job', 'education']] = imputer.fit_transform(X_train[['job', 'education']])
X_valid[['job', 'education']] = imputer.fit_transform(X_valid[['job', 'education']])
X_test[['job', 'education']]  = imputer.fit_transform(X_test[['job', 'education']])

In [87]:
cat_vars = list(set(X_train.columns).difference(num_vars))

for col in cat_vars:
  X_train[col] = X_train[col].astype("category")
  X_valid[col] = X_valid[col].astype("category")
  X_test[col]  = X_test[col].astype("category")

In [95]:
X_campaign_test = campaign_test.drop(columns=['job', 'education'])
X_imp_campaign_test = X_campaign_test[X_campaign_test['device'] == 'na'].drop(columns='device')
X_campaign_test = X_campaign_test[X_campaign_test['device'] != 'na']
y_campaign_test = X_campaign_test[['device']]
X_campaign_test = X_campaign_test.drop(columns='device')
y_campaign_test = enc.fit_transform(y_campaign_test)
y_imp_campaign_test = device_imputation.predict(X_imp_campaign_test)
campaign_test.loc[X_imp_campaign_test.index, 'device'] = y_imp_campaign_test
campaign_test.loc[campaign_test['device'] == 0, 'device'] = 'smartphone'
campaign_test.loc[campaign_test['device'] == 1, 'device'] = 'desktop'
campaign_test[['job', 'education']] = imputer.fit_transform(campaign_test[['job', 'education']])
for col in cat_vars:
  campaign_test[col] = campaign_test[col].astype("category")

# XGBoost

In [94]:
# xgb_clf = XGBClassifier(tree_method="hist", enable_categorical=True)
# param_search = {'n_estimators': [150, 200, 250],
#                 'learning_rate': [0.16, 0.17, 0.18, 0.19],
#                 'max_depth': range(2, 5),
#                 'colsample_bytree': [1],
#                 'gamma': [i/10.0 for i in range(1, 4)],
#                 'scale_pos_weight':[.7, .8, 1]}
# model = GridSearchCV(xgb_clf, scoring='accuracy', param_grid=param_search, return_train_score=True, verbose=1, cv=3)
# model.fit(X_train, y_train)

Fitting 3 folds for each of 324 candidates, totalling 972 fits


GridSearchCV(cv=3,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=True, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     gpu_id=None, grow_policy=None,
                                     importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None, m...
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs=None,
                                     num_parallel_tree=None, predictor=None,
                                     random_state=None, ...),
             param_grid={'colsample_bytree': [1], 'gamma': [0.1, 0.2, 0.3],
                         'learning_rate': [0.16, 0.17, 0.18, 0.19],
                         'max_depth': range(2, 5),
                         'n_estimators': [150, 200, 250],
                         'scale_pos_weight': [0.7, 0.8, 1]},
             return_train_score=True, scoring='accuracy', verbose=1)

In [97]:
best_params=model.best_params_
best_params

{'colsample_bytree': 1,
 'gamma': 0.1,
 'learning_rate': 0.17,
 'max_depth': 3,
 'n_estimators': 150,
 'scale_pos_weight': 1}

In [98]:
model.best_score_

0.8547727957732745

In [99]:
xgb_model = XGBClassifier(**best_params, tree_method='hist', enable_categorical=True)
xgb_model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=True,
              eval_metric=None, feature_types=None, gamma=0.1, gpu_id=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.17, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=150, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [137]:
train_pred = xgb_model.predict_proba(X_train)[:,1]
valid_pred = xgb_model.predict_proba(X_valid)[:,1]

In [138]:
cutoff_table = pd.DataFrame({'cutoff': 1e-2*np.arange(10, 95, 5)})
cutoff_table['train_acc'] = [np.round(accuracy_score(y_train, (train_pred > cutoff).astype(int)), 3)
                             for cutoff in cutoff_table['cutoff']]
cutoff_table['valid_acc'] = [np.round(accuracy_score(y_valid, (valid_pred > cutoff).astype(int)), 3)
                             for cutoff in cutoff_table['cutoff']]
cutoff_table

,cutoff,train_acc,valid_acc
0,0.10,0.788,0.768
1,0.15,0.832,0.797
2,0.20,0.856,0.817
3,0.25,0.874,0.830
4,0.30,0.886,0.844
5,0.35,0.896,0.844
6,0.40,0.900,0.841
7,0.45,0.902,0.841
8,0.50,0.902,0.833
9,0.55,0.904,0.826


In [141]:
cutoff_table = pd.DataFrame({'cutoff': 1e-2*np.arange(30, 50, 1)})
cutoff_table['train_acc'] = [np.round(accuracy_score(y_train, (train_pred > cutoff).astype(int)), 4)
                             for cutoff in cutoff_table['cutoff']]
cutoff_table['valid_acc'] = [np.round(accuracy_score(y_valid, (valid_pred > cutoff).astype(int)), 4)
                             for cutoff in cutoff_table['cutoff']]
cutoff_table

,cutoff,train_acc,valid_acc
0,0.30,0.8862,0.8436
1,0.31,0.8894,0.8421
2,0.32,0.8920,0.8421
3,0.33,0.8934,0.8407
4,0.34,0.8947,0.8429
5,0.35,0.8961,0.8436
6,0.36,0.8969,0.8399
7,0.37,0.8975,0.8407
8,0.38,0.8987,0.8414
9,0.39,0.8987,0.8407


Choosing 0.35 cutoff

In [145]:
treshold = 0.35

In [147]:
y_test_pred = (xgb_model.predict_proba(X_test)[:,1] > treshold).astype(int)
accuracy_score(y_test, y_test_pred)

0.8481012658227848

In [161]:
X = pd.concat([X_train, X_valid, X_test], axis=0)
y = pd.concat([y_train, y_valid, y_test], axis=0)
xgb_model.fit(X, y)
y_REAL_test = (xgb_model.predict_proba(campaign_test)[:,1] > treshold).astype(int)

In [ ]:
file = open('test_file_xgboost.csv', 'w')
writer = csv.writer(file)
writer.writerow(['Id', 'subscription'])
for i in range(len(y_REAL_test)):
    writer.writerow([i, y_REAL_test[i]])
file.close()